In [43]:
%set_env OPENAI_API_KEY

UsageError: usage is 'set_env var=val'


In [3]:
import requests
from pydantic import BaseModel, Field
import datetime
from langchain.tools import tool

# Define the input schema
class OpenMeteoInput(BaseModel):
    latitude: float = Field(..., description="Latitude of the location to fetch weather data for")
    longitude: float = Field(..., description="Longitude of the location to fetch weather data for")

@tool(args_schema=OpenMeteoInput)
def get_current_temperature(latitude: float, longitude: float) -> dict:
    """Fetch current temperature for given coordinates."""
    
    BASE_URL = "https://api.open-meteo.com/v1/forecast"
    
    # Parameters for the request
    params = {
        'latitude': latitude,
        'longitude': longitude,
        'hourly': 'temperature_2m',
        'forecast_days': 1,
    }

    # Make the request
    response = requests.get(BASE_URL, params=params)
    
    if response.status_code == 200:
        results = response.json()
    else:
        raise Exception(f"API Request failed with status code: {response.status_code}")

    current_utc_time = datetime.datetime.utcnow()
    time_list = [datetime.datetime.fromisoformat(time_str.replace('Z', '+00:00')) for time_str in results['hourly']['time']]
    temperature_list = results['hourly']['temperature_2m']
    
    closest_time_index = min(range(len(time_list)), key=lambda i: abs(time_list[i] - current_utc_time))
    current_temperature = temperature_list[closest_time_index]
    
    return f'The current temperature is {current_temperature}°C'

In [4]:
import wikipedia

@tool
def search_wikipedia(query: str) -> str:
    """Run Wikipedia search and get page summaries."""
    page_titles = wikipedia.search(query)
    summaries = []
    for page_title in page_titles[: 3]:
        try:
            wiki_page =  wikipedia.page(title=page_title, auto_suggest=False)
            summaries.append(f"Page: {page_title}\nSummary: {wiki_page.summary}")
        except (
            self.wiki_client.exceptions.PageError,
            self.wiki_client.exceptions.DisambiguationError,
        ):
            pass
    if not summaries:
        return "No good Wikipedia Search Result was found"
    return "\n\n".join(summaries)

In [5]:
tools = [get_current_temperature, search_wikipedia]

In [8]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.tools.render import format_tool_to_openai_function
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser

In [9]:
functions = [format_tool_to_openai_function(f) for f in tools]
model = ChatOpenAI(temperature=0).bind(functions=functions)
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful but sassy assistant"),
    ("user", "{input}"),
])
chain = prompt | model | OpenAIFunctionsAgentOutputParser()

In [10]:
result = chain.invoke({"input": "what is the weather is sf?"})

In [11]:
result

AgentActionMessageLog(tool='get_current_temperature', tool_input={'latitude': 37.7749, 'longitude': -122.4194}, log="\nInvoking: `get_current_temperature` with `{'latitude': 37.7749, 'longitude': -122.4194}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"latitude":37.7749,"longitude":-122.4194}', 'name': 'get_current_temperature'}}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 136, 'total_tokens': 161}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_3b956da36b', 'finish_reason': 'function_call', 'logprobs': None}, id='run-9687e662-482d-4939-a5a5-80ea11d2d462-0')])

In [13]:
from langchain.prompts import MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages(
    [
        ("system","you are helpful but sassy assistant"),
        ("user","{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad")
    ]
)



In [14]:
chain = prompt | model | OpenAIFunctionsAgentOutputParser()

In [16]:
result1 = chain.invoke({"input":"what is the weather in sf?","agent_scratchpad":[]})

In [17]:
result1.tool

'get_current_temperature'

In [18]:
result1.tool_input

{'latitude': 37.7749, 'longitude': -122.4194}

In [21]:
observation = get_current_temperature(result1.tool_input)
observation

'The current temperature is 9.9°C'

In [20]:
from langchain.agents.format_scratchpad import format_to_openai_functions

In [22]:
format_to_openai_functions([(result1, observation)])

[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"latitude":37.7749,"longitude":-122.4194}', 'name': 'get_current_temperature'}}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 136, 'total_tokens': 161}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_3b956da36b', 'finish_reason': 'function_call', 'logprobs': None}, id='run-f9eca479-3b60-4151-98f8-0728a1355dbf-0'),
 FunctionMessage(content='The current temperature is 9.9°C', name='get_current_temperature')]

In [24]:
chain.invoke({
    "input": "what is the weather is sf?", 
    "agent_scratchpad": format_to_openai_functions([(result1, observation)])
})

AgentFinish(return_values={'output': 'The current temperature in San Francisco is 9.9°C.'}, log='The current temperature in San Francisco is 9.9°C.')

In [25]:
from langchain.schema.agent import AgentFinish

In [28]:
def run_agent(user_input):
    intermediate_steps = []
    while True:
        result = chain.invoke({"input":user_input,"agent_scratchpad": format_to_openai_functions(intermediate_steps)})

        if isinstance(result,AgentFinish):
            return result
        tool = {
            "search_wikipedia": search_wikipedia, 
            "get_current_temperature": get_current_temperature,
        }[result.tool]
        observation = tool.run(result.tool_input)
        intermediate_steps.append((result,observation))
        

In [29]:
run_agent("what is the weather in hyderabad telangana india ")

AgentFinish(return_values={'output': 'The current temperature in Hyderabad, Telangana, India is 40.1°C. Stay cool!'}, log='The current temperature in Hyderabad, Telangana, India is 40.1°C. Stay cool!')

In [32]:
from langchain.schema.runnable import RunnablePassthrough
agent_chain = RunnablePassthrough.assign(
    agent_scratchpad =  lambda x : format_to_openai_functions(x['intermediate_steps'])
) | chain

In [33]:
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent_chain,tools=tools,verbose=True)

In [34]:
agent_executor.invoke({"input": "what is langchain?"})



> Entering new AgentExecutor chain...

Invoking: `search_wikipedia` with `{'query': 'Langchain'}`


Page: LangChain
Summary: LangChain is a framework designed to simplify the creation of applications using large language models (LLMs). As a language model integration framework, LangChain's use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis.

Page: Sentence embedding
Summary: In natural language processing, a sentence embedding refers to a numeric representation of a sentence in the form of a vector of real numbers which encodes meaningful semantic information.State of the art embeddings are based on the learned hidden layer representation of dedicated sentence transformer models. BERT pioneered an approach involving the use of a dedicated [CLS] token prepended to the beginning of each sentence inputted into the model; the final hidden state vector of this token encodes information about the se

{'input': 'what is langchain?',
 'output': 'LangChain is a framework designed to simplify the creation of applications using large language models (LLMs). It is a language model integration framework with use-cases including document analysis and summarization, chatbots, and code analysis.'}

In [35]:
########To remember history

In [36]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful but sassy assistant"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user","{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

In [37]:
agent_chain = RunnablePassthrough.assign(
    agent_scratchpad= lambda x: format_to_openai_functions(x["intermediate_steps"])
) | prompt | model | OpenAIFunctionsAgentOutputParser()

In [38]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(return_messages=True,memory_key="chat_history")

In [39]:
agent_executor = AgentExecutor(agent=agent_chain, tools=tools, verbose=True, memory=memory)

In [40]:
agent_executor.invoke({"input":"my name is Praveen Reddy C "})



> Entering new AgentExecutor chain...
Hello Praveen Reddy C! How can I assist you today?

> Finished chain.


{'input': 'my name is Praveen Reddy C ',
 'chat_history': [HumanMessage(content='my name is Praveen Reddy C '),
  AIMessage(content='Hello Praveen Reddy C! How can I assist you today?')],
 'output': 'Hello Praveen Reddy C! How can I assist you today?'}

In [41]:
agent_executor.invoke({"input":" what is the weather in munagala, itikyala, telangana "})



> Entering new AgentExecutor chain...

Invoking: `get_current_temperature` with `{'latitude': 17.3724, 'longitude': 78.4242}`


The current temperature is 40.3°CThe current temperature in Munagala, Itikyala, Telangana is 40.3°C. It's quite hot out there!

> Finished chain.


{'input': ' what is the weather in munagala, itikyala, telangana ',
 'chat_history': [HumanMessage(content='my name is Praveen Reddy C '),
  AIMessage(content='Hello Praveen Reddy C! How can I assist you today?'),
  HumanMessage(content=' what is the weather in munagala, itikyala, telangana '),
  AIMessage(content="The current temperature in Munagala, Itikyala, Telangana is 40.3°C. It's quite hot out there!")],
 'output': "The current temperature in Munagala, Itikyala, Telangana is 40.3°C. It's quite hot out there!"}

In [42]:
agent_executor.invoke({"input":"Can you guess my name ?"})



> Entering new AgentExecutor chain...
Hmm, let me try! Is your name Praveen Reddy C?

> Finished chain.


{'input': 'Can you guess my name ?',
 'chat_history': [HumanMessage(content='my name is Praveen Reddy C '),
  AIMessage(content='Hello Praveen Reddy C! How can I assist you today?'),
  HumanMessage(content=' what is the weather in munagala, itikyala, telangana '),
  AIMessage(content="The current temperature in Munagala, Itikyala, Telangana is 40.3°C. It's quite hot out there!"),
  HumanMessage(content='Can you guess my name ?'),
  AIMessage(content='Hmm, let me try! Is your name Praveen Reddy C?')],
 'output': 'Hmm, let me try! Is your name Praveen Reddy C?'}